# Tensor and Add Operation

ttnn.Tensor is the central type of ttnn.

It is similar to torch.Tensor in the sense that it represents multi-dimensional matrix containing elements of a single data type.

The are a few key differences:

- ttnn.Tensor can be stored in the SRAM or DRAM of Tenstorrent devices
- ttnn.Tensor doesn't have a concept of the strides, however it has a concept of row-major and tile layout
- ttnn.Tensor has support for data types not supported by torch such as `bfp8` for example
- ttnn.Tensor's shape stores the padding added to the tensor due to TILE_LAYOUT

## Creating a tensor

The recommended way to create a tensor is by using torch create function and then simply calling `ttnn.from_torch`. So, let's import both `torch` and `ttnn`

In [14]:
import torch
import ttnn

## Open the device

Use `ttnn.open` to get a handle to the device. In order to actually utilize the tensor, we need to put it on the device. So, that we can run `ttnn` operations on it

In [15]:
device_id = 0
device = ttnn.open_device(device_id=device_id)

2025-06-30 02:07:28.156 | info     |           Metal | Initializing device 0. Program cache is enabled (device.cpp:429)
2025-06-30 02:07:28.168 | info     |           Metal | Closing mesh device 1 (mesh_device.cpp:488)


And now let's create a torch Tensor and convert it to ttnn Tensor

In [16]:
ttnn_tensor = ttnn.rand((3, 4), device=device)

print(f"shape: {ttnn_tensor.shape}")
print(f"layout: {ttnn_tensor.layout}")
print(f"dtype: {ttnn_tensor.dtype}")

shape: Shape([3, 4])
layout: Layout.TILE
dtype: DataType.BFLOAT16


As expected we get a tensor of shape [3, 4] in row-major layout with a data type of float32.

## Host Storage: Borrowed vs Owned

In this particular case, ttnn Tensor will borrow the data of the torch Tensor because ttnn Tensor is in row-major layout, torch tensor is contiguous and their data type matches.

Let's print the current ttnn tensor, set element of torch tensor to 1234 and print the ttnn Tensor again to see borrowed storage in action

In [17]:
# ttnn_tensor = ttnn.rand((3, 4), dtype=ttnn.float32, device=device)
# borrowed_ttnn_tensor = ttnn.from_device(ttnn_tensor)
# print(f"Original values:\n{ttnn_tensor}")
# borrowed_ttnn_tensor[:] = 1234
# print(f"New values are all going to be 1234:\n{ttnn_tensor}")

We try our best to use borrowed storage but if the torch data type is not supported in ttnn, then we don't have a choice but to automatically pick a different data type and copy data

In [18]:
torch_tensor = ttnn.to_torch(ttnn.rand((3, 4), device=device))
ttnn_tensor = ttnn.from_torch(torch_tensor)
print("torch_tensor.dtype:", torch_tensor.dtype)
print("ttnn_tensor.dtype:", ttnn_tensor.dtype)

print(f"Original values:\n{ttnn_tensor}")
torch_tensor[0, 0] = 1234
#print(f"Original values again because the tensor doesn't use borrowed storage:\n{ttnn_tensor}")

torch_tensor.dtype: torch.bfloat16
ttnn_tensor.dtype: DataType.BFLOAT16
Original values:
ttnn.Tensor([[ 0.83984,  0.91797,  0.36914,  0.68359],
             [ 0.59766,  0.29883,  0.39844,  0.19922],
             [ 0.36328,  0.67969,  0.45898,  0.72656]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


## Data Type

The data type of the ttnn tensor can be controlled explicitly when conversion from torch.

In [19]:
torch_tensor = ttnn.to_torch(ttnn.rand((3, 4), dtype=ttnn.float32, device=device))
ttnn_tensor = ttnn.from_torch(torch_tensor, dtype=ttnn.bfloat16)
print(f"torch_tensor.dtype: {torch_tensor.dtype}")
print(f"ttnn_tensor.dtype: {ttnn_tensor.dtype}")

torch_tensor.dtype: torch.float32
ttnn_tensor.dtype: DataType.BFLOAT16


## Layout

Tenstorrent hardware is most efficiently utilized when running tensors using [tile layout](https://tenstorrent.github.io/ttnn/latest/ttnn/tensor.html#layout).
The current tile size is hard-coded to [32, 32]. It was determined to be the optimal size for a tile given the compute, memory and data transfer constraints.


ttnn provides easy and intuitive way to convert from row-major layout to tile layout and back.

In [20]:
ttnn_tensor = ttnn.rand((3, 4), layout=ttnn.ROW_MAJOR_LAYOUT, device=device)
print(f"Tensor in row-major layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")
ttnn_tensor = ttnn.to_layout(ttnn_tensor, ttnn.TILE_LAYOUT)
print(f"Tensor in tile layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")
ttnn_tensor = ttnn.to_layout(ttnn_tensor, ttnn.ROW_MAJOR_LAYOUT)
print(f"Tensor back in row-major layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")

Tensor in row-major layout:
Shape Shape([3, 4])
Layout: Layout.ROW_MAJOR
ttnn.Tensor([[1236992.00000,  0.86328, 1220608.00000,  0.43164],
             [ 0.78125,  0.39062,  0.13184,  0.56250],
             [ 0.00406,  0.50000,  0.01624,  0.00812]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)
Tensor in tile layout:
Shape Shape([3, 4])
Layout: Layout.TILE
ttnn.Tensor([[1236992.00000,  0.86328, 1220608.00000,  0.43164],
             [ 0.78125,  0.39062,  0.13184,  0.56250],
             [ 0.00406,  0.50000,  0.01624,  0.00812]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::TILE)
Tensor back in row-major layout:
Shape Shape([3, 4])
Layout: Layout.ROW_MAJOR
ttnn.Tensor([[1236992.00000,  0.86328, 1220608.00000,  0.43164],
             [ 0.78125,  0.39062,  0.13184,  0.56250],
             [ 0.00406,  0.50000,  0.01624,  0.00812]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


Note that padding is automatically inserted to put the tensor into tile layout and it automatically removed after the tensor is converted back to row-major layout

The conversion to tile layout can be done when caling `ttnn.from_torch`

In [21]:
torch_tensor = ttnn.to_torch(ttnn.rand((3, 4), device=device, layout=ttnn.ROW_MAJOR_LAYOUT))
ttnn_tensor = ttnn.from_torch(torch_tensor, layout=ttnn.TILE_LAYOUT)
print(f"Tensor in row-major layout:\nShape {torch_tensor.shape}; Layout: {torch_tensor.layout}")
print(f"Tensor in tile layout:\nShape {ttnn_tensor.shape}; Layout: {ttnn_tensor.layout}")

Tensor in row-major layout:
Shape torch.Size([3, 4]); Layout: torch.strided
Tensor in tile layout:
Shape Shape([3, 4]); Layout: Layout.TILE


Note that `ttnn.to_torch` will always convert to row-major layout

## Initialize tensors a and b with random values using torch

Create a tensor that can be used by a `ttnn` operation with the `ttnn.TILE_LAYOUT` and put the tensor on the `device`

In [22]:
input_tensor_a = ttnn.rand((32, 32), dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)
input_tensor_b = ttnn.rand((32, 32), dtype=ttnn.bfloat16, layout=ttnn.TILE_LAYOUT, device=device)

print(f"input_tensor_a:\n{input_tensor_a}")
print(f"input_tensor_b:\n{input_tensor_b}")

input_tensor_a:
ttnn.Tensor([[ 0.27344,  0.63672,  ...,  0.56250,  0.78125],
             [ 0.50781,  0.75391,  ...,  0.53125,  0.26562],
             ...,
             [ 0.05469,  0.52734,  ...,  0.50781,  0.75391],
             [ 0.50000,  0.25000,  ...,  0.19434,  0.09717]], shape=Shape([32, 32]), dtype=DataType::BFLOAT16, layout=Layout::TILE)
input_tensor_b:
ttnn.Tensor([[ 0.91016,  0.95312,  ...,  0.30859,  0.15430],
             [ 0.40430,  0.69922,  ...,  0.96875,  0.98438],
             ...,
             [ 0.95703,  0.47852,  ...,  0.40430,  0.69922],
             [ 0.55859,  0.77734,  ...,  0.78906,  0.89453]], shape=Shape([32, 32]), dtype=DataType::BFLOAT16, layout=Layout::TILE)


## Add tensor a and b

`ttnn` supports operator overloading, therefore operator `+` can be used instead of `torch.add`

In [23]:
output_tensor = input_tensor_a + input_tensor_b
print(f"output_tensor:\n{output_tensor}")

output_tensor:
ttnn.Tensor([[ 1.18750,  1.59375,  ...,  0.87109,  0.93750],
             [ 0.91406,  1.45312,  ...,  1.50000,  1.25000],
             ...,
             [ 1.01562,  1.00781,  ...,  0.91406,  1.45312],
             [ 1.06250,  1.03125,  ...,  0.98438,  0.99219]], shape=Shape([32, 32]), dtype=DataType::BFLOAT16, layout=Layout::TILE)


## Inspect the output tensor of the add in ttnn

As can be seen the tensor of the same shape, layout and dtype is produced

In [24]:
print(f"shape: {output_tensor.shape}")
print(f"dtype: {output_tensor.dtype}")
print(f"layout: {output_tensor.layout}")

shape: Shape([32, 32])
dtype: DataType.BFLOAT16
layout: Layout.TILE


In general we expect layout and dtype to stay the same when running most operations unless explicit arguments to modify them are passed in. However, there are obvious exceptions like an embedding operation that takes in `ttnn.uint32` and produces `ttnn.bfloat16`

## Convert to torch and inspect the attributes of the torch tensor

When converting the tensor to torch, `ttnn.to_torch` will move the tensor from the device, convert to tile layout and figure out the best data type to use on the torch side

In [25]:
output_tensor = ttnn.to_torch(output_tensor)
print(f"shape: {output_tensor.shape}")
print(f"dtype: {output_tensor.dtype}")

shape: torch.Size([32, 32])
dtype: torch.bfloat16


## Close the device

Close the handle the device. This is a very important step as the device can hang currently if not closed properly

In [26]:
ttnn.close_device(device)

2025-06-30 02:07:28.795 | info     |           Metal | Closing mesh device 3 (mesh_device.cpp:488)
2025-06-30 02:07:28.797 | info     |           Metal | Closing mesh device 2 (mesh_device.cpp:488)
2025-06-30 02:07:28.797 | info     |           Metal | Closing device 0 (device.cpp:469)
2025-06-30 02:07:28.797 | info     |           Metal | Disabling and clearing program cache on device 0 (device.cpp:781)
